In [1]:
import numpy as np
import nibabel
import nilearn
from nilearn.image import resample_to_img
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.plotting import show
from nilearn.plotting import plot_roi
from nilearn import image
from nilearn.masking import apply_mask
# get_ipython().magic('matplotlib inline')
import scipy
import matplotlib
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.input_data import NiftiMasker
#from nilearn import plotting
import nibabel
from nilearn.masking import apply_mask
from nilearn.image import load_img
from nilearn.image import new_img_like
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, svm, metrics
from sklearn.linear_model import Ridge
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import KFold
from sklearn.cross_validation import LeaveOneLabelOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_selection import SelectFwe
from scipy import signal
from scipy.fftpack import fft, fftshift
from scipy import interp

params = {'legend.fontsize': 'large',
          'figure.figsize': (5, 3),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
font = {'weight': 'bold',
        'size': 22}
plt.rc('font', **font)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, f_classif, GenericUnivariateSelect, SelectKBest, chi2
from sklearn.feature_selection import RFE
import os
import seaborn as sns
import pandas as pd
import csv
from scipy import stats
import brainiak
import brainiak.funcalign.srm
import glob

/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
projectDir='/jukebox/norman/amennen/prettymouth/'
DMNmask='/jukebox/norman/amennen/MNI_things/Yeo_JNeurophysiol11_MNI152/Yeo_Network7mask_reoriented_resampledBOLD.nii.gz'
fmriprep_dir=projectDir + '/derivatives/fmriprep'

In [3]:
# load subject numbers
subInd = 0
nsub=38
allnames = []
allgroups = []
groupInfo={}
# skip subjects 039 and 116
with open(projectDir + 'participants.tsv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        if 'sub' in row[0]:
            # now skip the subjects we don't want to analyze
            allInfo = row[0].split('\t')
            subjName=allInfo[0]
            if subjName != 'sub-039' and subjName != 'sub-116':
                if allInfo[3] == 'paranoia':
                    group = 0
                elif allInfo[3] == 'affair':
                    group = 1
                allnames.append(subjName)
                allgroups.append(group)
                subInd+=1

In [4]:
paranoidSubj = allnames[0:19]
cheatingSubj = allnames[19:]
paranoidLabel = allgroups[0:19]
cheatingLabel = allgroups[19:]
nfolds=19
print(allnames)

['sub-088', 'sub-089', 'sub-090', 'sub-091', 'sub-092', 'sub-093', 'sub-094', 'sub-095', 'sub-096', 'sub-097', 'sub-098', 'sub-099', 'sub-100', 'sub-101', 'sub-102', 'sub-103', 'sub-104', 'sub-105', 'sub-106', 'sub-107', 'sub-108', 'sub-109', 'sub-110', 'sub-111', 'sub-068', 'sub-081', 'sub-112', 'sub-053', 'sub-113', 'sub-031', 'sub-114', 'sub-115', 'sub-117', 'sub-118', 'sub-119', 'sub-120', 'sub-121', 'sub-122']


In [41]:
k1=10
k2=200
filename = 'accuracy_SRM_randomized/accuracy_k1_' + str(k1) + '_k2_' + str(k2) + '_rep_*storyTRz.npy'
all_file_names = glob.glob(filename)
n_iter = len(all_file_names)
accuracy = np.zeros(n_iter)
print(n_iter)
for f in np.arange(n_iter):
    z = np.load(all_file_names[f])
    accuracy[f] = z[0]
print('*********')
print('average accuracy is for k1 %i and k2 %i: %2.2f +/- %2.2f' % (k1,k2,np.mean(accuracy),np.std(accuracy)))


1000
*********
average accuracy is for k1 10 and k2 200: 0.57 +/- 0.35


In [125]:
## HERE: WE'RE SPLITTING TRS BY LOW/HIGH DIFFERENCE
k1=20
k2=100
lowhigh = 1
if lowhigh == 1:
    filename = 'accuracy_SRM_randomized/accuracy_LOW_k1_' + str(k1) + '_k2_' + str(k2) + '_rep_*storyTRz.npy'
elif lowhigh == 2:
    filename = 'accuracy_SRM_randomized/accuracy_HIGH_k1_' + str(k1) + '_k2_' + str(k2) + '_rep_*storyTRz.npy'
all_file_names = glob.glob(filename)
n_iter = len(all_file_names)
accuracy = np.zeros(n_iter)
print(n_iter)
for f in np.arange(n_iter):
    z = np.load(all_file_names[f])
    accuracy[f] = z[0]
print('*********')
print('average accuracy is for k1 %i and k2 %i: %2.2f +/- %2.2f' % (k1,k2,np.mean(accuracy),np.std(accuracy)))


1000
*********
average accuracy is for k1 20 and k2 100: 0.54 +/- 0.36


In [31]:
## HERE: WE'RE SPLITTING TRS BY LOW/HIGH DIFFERENCE
k1=1
k2=200
lowhigh = 2
if lowhigh == 1:
    filename = 'accuracy_SRM_randomized/accuracy_LOW_k1_' + str(k1) + '_k2_' + str(k2) + '_rep_*storyTRz.npy'
elif lowhigh == 2:
    filename = 'accuracy_SRM_randomized/accuracy_HIGH_k1_' + str(k1) + '_k2_' + str(k2) + '_rep_*storyTRz.npy'
all_file_names = glob.glob(filename)
n_iter = len(all_file_names)
accuracy = np.zeros(n_iter)
print(n_iter)
for f in np.arange(n_iter):
    z = np.load(all_file_names[f])
    accuracy[f] = z[0]
print('*********')
print('average accuracy is for k1 %i and k2 %i: %2.2f +/- %2.2f' % (k1,k2,np.mean(accuracy),np.std(accuracy)))


1000
*********
average accuracy is for k1 1 and k2 200: 0.73 +/- 0.29


In [10]:
## HERE: WE'RE SPLITTING TRS BY LOW/HIGH DIFFERENCE 
## HERE IT'S THE HELD OUT DATA!!!
k1=1
k2=100
lowhigh = 1
if lowhigh == 1:
    filename = 'accuracy_SRM_randomized/nosubject_accuracy_LOW_k1_' + str(k1) + '_k2_' + str(k2) + '_rep_*storyTRz.npy'
elif lowhigh == 2:
    filename = 'accuracy_SRM_randomized/nosubject_accuracy_HIGH_k1_' + str(k1) + '_k2_' + str(k2) + '_rep_*storyTRz.npy'
all_file_names = glob.glob(filename)
n_iter = len(all_file_names)
accuracy = np.zeros(n_iter)
print(n_iter)
for f in np.arange(n_iter):
    z = np.load(all_file_names[f])
    accuracy[f] = z[0]
print('*********')
print('average accuracy is for k1 %i and k2 %i: %2.2f +/- %2.2f' % (k1,k2,np.mean(accuracy),np.std(accuracy)))


687
*********
average accuracy is for k1 1 and k2 100: 0.75 +/- 0.27


In [5]:
## HERE: WE'RE SPLITTING TRS BY LOW/HIGH DIFFERENCE
k1=0
k2=0
lowhigh = 2
if lowhigh == 1:
    filename = 'accuracy_SRM_randomized/nosubject_accuracy_LOW_k1_' + str(k1) + '_k2_' + str(k2) + '_rep*storyTRz.npy'
elif lowhigh == 2:
    filename = 'accuracy_SRM_randomized/nosubject_accuracy_HIGH_k1_' + str(k1) + '_k2_' + str(k2) + '_rep*storyTRz.npy'
all_file_names = glob.glob(filename)
n_iter = len(all_file_names)
accuracy = np.zeros(n_iter)
print(n_iter)
for f in np.arange(n_iter):
    z = np.load(all_file_names[f])
    accuracy[f] = z[0]
print('*********')
print('average accuracy is for k1 %i and k2 %i: %2.2f +/- %2.2f' % (k1,k2,np.mean(accuracy),np.std(accuracy)))


1000
*********
average accuracy is for k1 0 and k2 0: 0.92 +/- 0.20


In [25]:
n_iter

884